# Eligiendo el número de clusters para k-means

## El método del codo

Cargamos las librerías necesarias

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
%matplotlib inline

Generamos un dataset y lo mostramos

In [0]:
dataset ,_ = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  # For reproducibility

In [0]:
plt.figure()
plt.title('Dataset')
plt.scatter(dataset[:, 0], dataset[:, 1],s=5)
plt.xticks(())
plt.yticks(())
plt.show()

In [0]:
costs = []
for i in range(1,20):
    est = KMeans(n_clusters=i)
    est.fit(dataset)
    costs.append(est.inertia_)

plt.plot(range(1,20),costs,'ro')
plt.axis([0,20,0,3500])
plt.show()

## El método de las siluetas

Al igual que antes comenzamos cargando las librerías necesarias

In [0]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

Vamos a generar las siluetas para el rango de clústeres de 2 a 9

In [0]:
range_n_clusters = range(2,9)

Generamos los diagramas utilizando la siguiente función que toma el dataset y el número de clústeres. Se generan dos figuras, a la izquierda el gráfico de siluetas y a la derecha la distribución de clústeres

In [0]:
def grafico_silueta(dataset,n_clusters):
        X = dataset
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(18, 7)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
        cluster_labels = clusterer.fit_predict(X)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            cmap = cm.get_cmap("Spectral")
            color = cmap(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        cmap = cm.get_cmap("Spectral")
        colors = cmap(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')

        # Labeling the clusters
        centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
        ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                    c="white", alpha=1, s=200, edgecolor='k')

        for i, c in enumerate(centers):
            ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                        s=50, edgecolor='k')

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                      "with n_clusters = %d, silhouete value %f" % (n_clusters,silhouette_avg)),
                     fontsize=14, fontweight='bold')

        
        plt.savefig("%d-kmeans.jpg" % n_clusters)

        plt.show()
        

Vemos el resultado para los distintos valores de k.

In [0]:
grafico_silueta(dataset,2)

In [0]:
grafico_silueta(dataset,3)

In [0]:
grafico_silueta(dataset,4)

In [0]:
grafico_silueta(dataset,5)

In [0]:
grafico_silueta(dataset,6)

In [0]:
grafico_silueta(dataset,7)

In [0]:
grafico_silueta(dataset,8)

Como se puede observar, el valor de silueta más alto se obtiene con k=4, seguido muy de cerca por k=5. Por lo tanto estos son los valores óptimos.  